In [3]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("tomatoHigh-prices.csv")

# 숫자형 변환
for col in ['평균가격', '전일', '전년']:
    df[col] = df[col].replace(',', '', regex=True).astype(int)

# DATE 변환
df['DATE'] = pd.to_datetime(df['DATE'])

# 빠진 날짜 채우기 (전체 날짜 생성 후 merge)
full_dates = pd.DataFrame({'DATE': pd.date_range(start=df['DATE'].min(), end=df['DATE'].max())})
df = pd.merge(full_dates, df, on='DATE', how='left')

# 파생변수: 평균가격 == 0 이거나 NaN이면 차이, 비율도 0
df['평균가격'] = df['평균가격'].fillna(0).astype(int)
df['전일'] = df['전일'].fillna(0).astype(int)
df['전일대비_가격차이'] = df.apply(
    lambda row: 0 if row['평균가격'] == 0 else row['평균가격'] - row['전일'],
    axis=1
)
df['전일대비_가격변화율(%)'] = df.apply(
    lambda row: 0 if row['평균가격'] == 0 or row['전일'] == 0 else (row['전일대비_가격차이'] / row['전일']) * 100,
    axis=1
)

# 공휴일 리스트 (2025 주요 공휴일)
holiday_list = [
    '2025-01-01', '2025-02-28', '2025-03-01', '2025-05-05', '2025-06-06',
    '2025-08-15', '2025-09-06', '2025-09-07', '2025-09-08', '2025-10-03',
    '2025-10-09', '2025-12-25'
]
holidays_kr_2025 = pd.to_datetime(holiday_list)

# 공휴일, 일요일 여부
df['공휴일여부'] = df['DATE'].isin(holidays_kr_2025).astype(int)
df['일요일여부'] = df['DATE'].dt.weekday.apply(lambda x: 1 if x == 6 else 0)

# 결과 확인
print(df[['DATE', '평균가격', '전일대비_가격차이', '전일대비_가격변화율(%)', '공휴일여부', '일요일여부']].head(10))

# 저장하려면
df.to_csv("tomatoHigh-prices-filled.csv", index=False)


        DATE   평균가격  전일대비_가격차이  전일대비_가격변화율(%)  공휴일여부  일요일여부
0 2020-04-09  16905      16905       0.000000      0      0
1 2020-04-10  15586      15586       0.000000      0      0
2 2020-04-11  14286      -1300      -8.340819      0      0
3 2020-04-12      0          0       0.000000      0      1
4 2020-04-13  14771      14771       0.000000      0      0
5 2020-04-14  14359       -412      -2.789249      0      0
6 2020-04-15  13052      -1307      -9.102305      0      0
7 2020-04-16  13177        125       0.957708      0      0
8 2020-04-17  13056       -121      -0.918267      0      0
9 2020-04-18  12462       -594      -4.549632      0      0


In [4]:
import pandas as pd

# 1. 파일 불러오기
price_df = pd.read_csv("tomatoHigh-prices-filled.csv")
amount_df = pd.read_csv("tomato_amount.csv")

# 2. 날짜 포맷 통일
price_df['DATE'] = pd.to_datetime(price_df['DATE'])
amount_df['DATE'] = pd.to_datetime(amount_df['DATE'])

# 3. 총반입량만 추출
amount_df_use = amount_df[['DATE', '총반입량']].copy()
amount_df_use['총반입량'] = amount_df_use['총반입량'].fillna(0).astype(int)

# 4. 날짜 기준 병합
df = pd.merge(price_df, amount_df_use, on='DATE', how='left')
df['총반입량'] = df['총반입량'].fillna(0).astype(int)

# 5. 전일대비 반입량 차이 및 변화율 계산
df['전일대비_반입량차이'] = df['총반입량'].diff().fillna(0).astype(int)
df['전일대비_반입량변화율(%)'] = df.apply(
    lambda row: 0 if row['총반입량'] == 0 or row['총반입량'] - row['전일대비_반입량차이'] == 0
    else (row['전일대비_반입량차이'] / (row['총반입량'] - row['전일대비_반입량차이'])) * 100,
    axis=1
)

# 6. 전일 or 전년이 0인 행들만 확인
zero_check = df[(df['전일'] == 0) | (df['전년'] == 0)]
print("전일 또는 전년이 0인 행 (예시):")
print(zero_check[['DATE', '평균가격', '전일', '전년']].head(10))

# 7. 평균가격·전일·전년이 모두 0인 행 제거 (필요시)
df_cleaned = df[~((df['평균가격'] == 0) & (df['전일'] == 0) & (df['전년'] == 0))]

# 8. 결과 저장 (선택)
# df_cleaned.to_csv("토마토_최종_정리본.csv", index=False)

# ✅ 확인용 출력
print("\n정리된 데이터 미리보기:")
print(df_cleaned[['DATE', '평균가격', '전일', '전년', '총반입량', '전일대비_반입량차이']].head(10))


전일 또는 전년이 0인 행 (예시):
         DATE   평균가격     전일       전년
0  2020-04-09  16905      0      0.0
1  2020-04-10  15586      0      0.0
4  2020-04-13  14771      0  11166.0
6  2020-04-15  13052  14359      0.0
11 2020-04-20  12872      0  11283.0
13 2020-04-22  13851  11757      0.0
18 2020-04-27  12293      0   9076.0
20 2020-04-29  11922  11299      0.0
23 2020-05-02  10370      0      0.0
24 2020-05-03      0      0      NaN

정리된 데이터 미리보기:
        DATE   평균가격     전일       전년  총반입량  전일대비_반입량차이
0 2020-04-09  16905      0      0.0   171           0
1 2020-04-10  15586      0      0.0   266          95
2 2020-04-11  14286  15586  11024.0   194         -72
3 2020-04-12      0  14286  11342.0     0        -194
4 2020-04-13  14771      0  11166.0   313         313
5 2020-04-14  14359  14771  10954.0   202        -111
6 2020-04-15  13052  14359      0.0   188         -14
7 2020-04-16  13177  13052  11157.0   186          -2
8 2020-04-17  13056  13177  11330.0   244          58
9 2020-04-18  124